In [1]:
import os
import numpy as np
from dotenv import load_dotenv

from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

## A.　載入 .env 取得 LLM API Key

In [2]:
load_dotenv()
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

C:\Users\PC\AppData\Local\Temp\ipykernel_24664\3523237785.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))


# 

## B.　RAG 實作

### I.　建立向量資料庫

In [3]:
documents = [
    "GPT-4 是 OpenAI 開發的最新語言模型，具備強大的自然語言處理能力。",
    # "RAG（檢索增強生成）結合了檢索與 AI 生成，提升模型回答的準確度。",
    "RAG是知名品牌製作的高效清潔抹布，通常市售30TWD，基本每戶人家都會有一條RAG放置廚房做使用",
    "FAISS 是一款高效的向量搜尋庫，適合用於 AI 相關應用。",
]

In [4]:
# 拆分文本為小塊
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docs = [Document(page_content=text) for text in text_splitter.split_text("\n".join(documents))]

# 產生嵌入並存入 FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

# 儲存 FAISS 索引
vectorstore.save_local("faiss_index")

print("✅ FAISS 向量資料庫已建立！")

✅ FAISS 向量資料庫已建立！


### II.　使用 RAG 進行增強式答覆

In [5]:
### 從 FAISS 檢索最相關的資料
def retrieve_from_faiss(query, top_k=2):
    vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})
    relevant_docs = retriever.get_relevant_documents(query)
    return [doc.page_content for doc in relevant_docs]  # 返回最相關的文本

In [6]:
### RAG 流程：檢索 + 生成回答
model = "gpt-4o"

def rag_query(query):
    relevant_docs = retrieve_from_faiss(query)
    context = "\n".join(relevant_docs)

    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 請根據以下資訊回答問題 : {context}
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    res = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    
    return res.choices[0].message.content

def norm_query(query):
    prompt = f"""
    我希望你扮演 "基於事實回答的顧問" 的角色。
    - 問題 : {query}
    - 遵守下列規則 : 
      * 請勿幻覺答覆
      * 繁體中文回覆
    """
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    res = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    return res.choices[0].message.content

# 

## C.　成果展示

### I.　什麼是 RAG？

In [7]:
norm_query("什麼是 RAG？")

'RAG 是「Retrieve and Generate」的縮寫，是一種結合檢索和生成技術的系統。這種方法主要用於自然語言處理，尤其是在回答問題和對話生成中。RAG 系統會先從一個大型資料庫或知識庫中檢索相關的信息，然後使用生成模型（如 GPT-3）對檢索到的信息進行加工和生成，給出最終的回答。這種方法可以提高回答的準確性和可靠性，因為它結合了檢索系統的資料完整性和生成模型的語言能力。這種技術在各種應用中都有廣泛的使用，例如智能問答系統和虛擬助手。'

In [8]:
rag_query("什麼是 RAG？")

C:\Users\PC\AppData\Local\Temp\ipykernel_24664\3191474142.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(query)


'根據您提供的資訊，RAG 是一個知名品牌，專門製作高效的清潔抹布，通常市價為30 TWD，基本上每戶人家都會在廚房中放置一條 RAG 清潔抹布使用。'

### II.　RAG 是抹布嗎? 超商可以買到它嗎?

In [9]:
norm_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'RAG 這個詞在日常生活中通常指的是抹布，是用來清潔表面的布料。在台灣的大部分超市或家居用品店，您應該能夠買到各種材質和大小的抹布。如果您指的是這種用途的 RAG，那麼在超市是可以找到的。 \n\n然而，RAG 也可能在其他語境下指代不同的詞彙，例如機構、組織名稱等。因此，具體需要根據上下文來確認。'

In [10]:
rag_query("RAG 是抹布嗎? 超商可以買到它嗎?")

'根據提供的信息，RAG 是一個知名品牌製作的高效清潔抹布，通常每戶人家都會在廚房放置一條來使用。至於是否能在超商買到，根據普遍的產品購買管道，這類常用的清潔抹布通常可以在一般家居用品店或超市購買，但具體是否能在每個超商買到可能會因地區或超商品牌的不同而有所差異。建議可以直接詢問當地超商或查看他們的商品目錄以確認是否有售。'

### III.　什麼是 Agent？

In [11]:
norm_query("什麼是 Agent？")

'在不同的領域中，「Agent」可以有不同的意義：\n\n1. **計算機科學與人工智慧**：在這些領域中，代理（Agent）通常指一種能夠感知其環境並採取行動以達成某些目標的軟體實體。多代理系統（MAS）是指多個這樣的代理相互協作來解決問題的系統。\n\n2. **經濟學**：在經濟學中，代理是指代表他人進行某種活動的人或企業，例如交易者、經紀人或其他中介機構。\n\n3. **化學與製藥**：在這些領域中，代理指的是引起化學反應或生理作用的物質。\n\n4. **法律**：在法律上，代理是指接受委託代表委託人處理事項的人，通常是在合同關係中。\n\n有哪些需要更深入了解的方面，可以根據具體領域再進一步解釋。'

In [12]:
rag_query("什麼是 Agent？")

'Agent 是指能夠在特定環境中感知、行動和進行決策的實體。它可以是軟體或硬體，在人工智慧領域特別常被提及。AI Agent 通常指的是具備自動化能力的程式，可以根據輸入數據、自身目標和環境狀況做出行動決策，以達成特定任務或最大化效能。不過，具體的實現範疇可能依領域而異，例如在機器學習、計算機科學或機器人領域中，Agent 的能力和功能都可能有不同的側重點。'